<a href="https://colab.research.google.com/github/MOOwuttichai/BSC_DPDM2023/blob/main/%E0%B8%A7%E0%B8%B4%E0%B9%80%E0%B8%84%E0%B8%A3%E0%B8%B2%E0%B8%B0%E0%B8%AB%E0%B9%8C%E0%B8%82%E0%B9%89%E0%B8%AD%E0%B8%A1%E0%B8%B9%E0%B8%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install pythainlp
!pip install -U sentence-transformers

In [23]:
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import Tokenizer
from pythainlp.util import normalize
from pythainlp.corpus.common import thai_words
from sklearn.cluster import KMeans
import pandas as pd

In [24]:
kok = pd.read_csv('/content/drive/MyDrive/เชี่ยงใหม่/Data_Docter_ศ.นพ.ชวลิต(หลังสกัด)_ใหม่!!.csv',encoding='utf-8')

In [25]:
kok

,name,comments,ยอดไลค์,จำนวนการตอบกลับ,จำนวนคำ,token,defind_cancer_with_nlp,use_ful,defind_exp_with_python,defind_Genden_with_nlp,...,อุจจาระเป็นเลือด,อ่อนเพลีย,เจ็บปวด,เบื่ออาหาร,เสียงแหบ,เหนื่อยง่าย,แผลเรื้อรัง,ไข้เรื้อรัง,ไม่มีการระบุอาการ,ไอเรื้อรัง
0,99แคมป์,ไม่ทานเนื้อค่ะ เห็นญาติพี่น้องรอบข้างเป็นมะ เ...,0,0,14,"['ไม่', 'ทาน', 'เนื้อ', 'ค่ะ', ' ', 'เห็น', 'ญ...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,ไม่ระบุเพศ,...,0,0,0,0,0,0,0,0,1,0
1,A Achi Sarakam,ปี62 แม่เกร็ดเลือดต่ำ+ค่าตับผิดปกติ ตรวจพบไวรั...,0,0,96,"['ปี', '62', ' ', 'แม่', 'เกร็ด', 'เลือด', 'ต่...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,เล่าประสบการณ์คนอื่น,เพศหญิง,...,0,0,0,0,0,0,0,0,1,0
2,A-Chiraya Kasiyapong,Bureerat Saisudta,0,0,4,"['Bureerat', ' ', 'Saisudta', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เพศหญิง,...,0,0,0,0,0,0,0,0,1,0
3,AI'Jack TheRam,ดีมากๆเลยครับ จะได้เป็นวิทยาทานให้คนอื่น พ่อกั...,0,0,89,"['ดีมาก', 'ๆ', 'เลย', 'ครับ', ' ', 'จะ', 'ได้'...",มะเร็งเต้านม,non,เล่าประสบการณ์คนอื่น,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
4,AS Kate-Duan,.,0,0,2,"['.', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,ไม่ว่าเจ้าไปยังที่ใด เจ้าก็ปรากฏตัวอยู่ตรงที่นั้น,กฤษมภัณฑ์ แสนเสมอ,0,0,6,"['กฤษม', 'ภัณฑ์', ' ', 'แสน', 'เสมอ', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
1606,ไอ มิส ยู,อันนี้ไม่รู้จริงๆนะคะ เราไม่สามารถตรวจครั้งแรก...,0,1,52,"['อันนี้', 'ไม่', 'รู้', 'จริงๆ', 'นะคะ', ' ',...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
1607,ไอด้า ไอรดาา,สมปอง มานุ,0,0,6,"['สม', 'ปอง', ' ', 'มา', 'นุ', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
1608,杨小龙,คุณแม่พึ่งพบว่าเป็นมะเร็งรังไข่ตอนตุลาคมปีนี้ค...,0,1,84,"['คุณแม่', 'พึ่ง', 'พบ', 'ว่า', 'เป็น', 'มะเร็...",มะเร็งรังไข่,non,เล่าประสบการณ์คนอื่น,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,0,0


In [26]:
len(kok[kok['defind_cancer_with_nlp']!='ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น'])

398

In [27]:
a=kok[kok['defind_cancer_with_nlp']!='ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น']

In [28]:
data = pd.read_csv('/content/drive/MyDrive/เชี่ยงใหม่/Data_Wiraon Chuenchom(หลังสกัด).csv',encoding='utf-8')

In [29]:
data

,name,comments,ยอดไลค์,จำนวนการตอบกลับ,จำนวนคำ,token,defind_cancer_with_nlp,use_ful,defind_exp_with_python,defind_Genden_with_nlp,...,อุจจาระเป็นเลือด,อ่อนเพลีย,เจ็บปวด,เบื่ออาหาร,เสียงแหบ,เหนื่อยง่าย,แผลเรื้อรัง,ไข้เรื้อรัง,ไม่มีการระบุอาการ,ไอเรื้อรัง
0,Aea Srilunchang,ขอแชร์เรื่องราวของแม่นะคะ (ตอนนี้กำลังทำการเข้...,0,0,36,"['ขอ', 'แชร์', 'เรื่องราว', 'ของ', 'แม่', 'นะค...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,เล่าประสบการณ์คนอื่น,เพศหญิง,...,0,0,0,0,0,0,0,0,1,0
1,Aekkarat Prayoonwong,Mai Chiangmai ความรู้ดี มีประโยชน์,0,0,9,"['Mai', ' ', 'Chiangmai', ' ', 'ความ', 'รู้ดี'...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เพศหญิง,...,0,0,0,0,0,0,0,0,1,0
2,Aemmi Ploy,ลูกพี่ลูกน้องเรา ตอนอายุ 28-29 ปีเป็นมะเร็งเต้...,0,0,47,"['ลูกพี่ลูกน้อง', 'เรา', ' ', 'ตอน', 'อายุ', '...",มะเร็งเต้านม,non,เล่าประสบการณ์คนอื่น,เพศหญิง,...,0,0,0,0,0,0,0,0,1,0
3,Alisa Deesirisathien,ตรวจพบว่าเป็นมะเร็งเต้านมระยะ 2 ตอนอายุ 28 ค่ะ...,14,1,217,"['ตรวจ', 'พบ', 'ว่า', 'เป็น', 'มะเร็ง', 'เต้าน...",มะเร็งเต้านม,non,เล่าประสบการณ์ตัวเอง,เพศหญิง,...,0,0,0,0,0,0,0,0,0,0
4,Amice A Little Angoon,อายุ 33 ไม่สูบบุหรี่ ไม่ดื่มเล่า แต่เครียดและน...,0,0,86,"['อายุ', ' ', '33', ' ', 'ไม่', 'สูบบุหรี่', '...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,เล่าประสบการณ์ตัวเอง,เพศหญิง,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,เหม่ยลี่ แม่เลี้ยงเดี่ยว,ขออนุญาต แปลประสบการณ์ ที่แม่เป็นมะเร็งตับ แม่...,0,0,78,"['ขออนุญาต', ' ', 'แปล', 'ประสบการณ์', ' ', 'ท...",มะเร็งตับ,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
288,แพร สุกัญญา,พ่อป่วยมะเร็งตับ ปวดท้องเหมือนเป็นกรดไหลย้อนค่...,10,5,21,"['พ่อ', 'ป่วย', 'มะเร็ง', 'ตับ', ' ', 'ปวดท้อง...",เล่ามากกว่า 2 โรค,non,เล่าประสบการณ์คนอื่น,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,0,0
289,แม่ยอดขมองอิ่ม' เมไร,สวัสดีค่ะ ตอนนั้นอายุ27ปีืเป็นมะเร็งปากมดลูกระ...,0,0,75,"['สวัสดี', 'ค่ะ', ' ', 'ตอนนั้น', 'อายุ', '27'...",มะเร็งปากมดลูก,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,1,0,0,0,0,0,0,0
290,ไพบูลย์ จณะวัตร,ภรรยาผมเป็นมะเร็งรังไข่ ตอนตรวจพบ ระยะ 4a มีอา...,0,0,81,"['ภรรยา', 'ผม', 'เป็น', 'มะเร็ง', 'รังไข่', ' ...",มะเร็งรังไข่,non,เล่าประสบการณ์ตัวเอง,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,0,0


In [30]:
comment=data.groupby('name').sum().reset_index()
# สร้าง set ข้อมูลภาษาไทย
words = set(thai_words())
words.remove("โรคมะเร็ง")
name =['กระเพาะปัสสวะ','กระเพาะปัสสาวะ','เยื่อบุโพรงมดลูก','ปากมดลูก','เม็ดเลือดขาว','กระเพาะอาหาร','กระเพราะอาหาร','ต่อมไทรอยด์','ต่อมไทยรอยด์','ท่อน้ำดี']
for i in name:
    words.add(i)
def remove_stopthai(tokens):
    final = [word.lower()
            for word in tokens if word not in thai_stopwords()]
    return final
def remove_bark(tokens):
    final = [word.lower() for word in tokens if word not in [' ','(',')','@','#','-','...','.','=','+','..','1','2','3','4','5','6','7','8','9','0']]
    return final
# สร้าง list เก็บตัว nlp เพิ่อนำไปวิเคราะห์โรค อาการ เเละเพศ
list_token =[]
count = []
for i in range(len(comment)):#len(comment)
    text= comment['comments'][i]
    custom_tokenizer = Tokenizer(words)
    Token = custom_tokenizer.word_tokenize(normalize(str(text)))
    Token.append('end')
    Token_final = remove_bark(remove_stopthai(Token))
    list_token.append(Token_final)
    count.append(len(Token_final))
comment['จำนวนคำ'] = count
comment['token'] = list_token

In [31]:
set_cancer=list(set(comment['defind_cancer_with_nlp']))

In [32]:
set_cancer

['มะเร็งหลอดอาหาร',
 'มะเร็งท่อน้ำดี',
 'มะเร็งไต',
 'มะเร็งกระเพาะปัสสาวะ',
 'มะเร็งตับ',
 'มะเร็งต่อมลูกหมาก',
 'มะเร็งรังไข่',
 'มะเร็งเม็ดเลือดขาว',
 'มะเร็งช่องปาก',
 'มะเร็งปากมดลูก',
 'มะเร็งผิวหนัง',
 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น',
 'มะเร็งสมอง',
 'เล่ามากกว่า 2 โรค',
 'มะเร็งลำไส้',
 'มะเร็งปอด',
 'มะเร็งเต้านม',
 'มะเร็งลำไส้ใหญ่',
 'มะเร็งต่อมน้ำเหลือง',
 'มะเร็งมดลูก',
 'มะเร็งถุงน้ำดี',
 'มะเร็งกระเพาะปัสสวะ',
 'มะเร็งเยื่อบุโพรงมดลูก',
 'มะเร็งตับอ่อน',
 'มะเร็งกระดูก',
 'มะเร็งกระเพาะอาหาร']

In [33]:
from nltk.probability import FreqDist
import plotly.graph_objects as go
import plotly.express as px
lung_cancer=comment[comment['defind_cancer_with_nlp']=='ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น'].reset_index()['token']
sum_word = []
for i in range(len(lung_cancer)):
    sum_word+=lung_cancer[i]
    sum_word.remove('end')
sum_word
fdist = FreqDist(sum_word)
x=fdist.most_common(10)
word_count_table = pd.DataFrame(x,columns=['word','count'])

In [34]:
lung_cancer

,token
0,"[แชร์, เรื่องราว, แม่, นะคะ, ตอนนี้, ทำการ, คี..."
1,"[mai, chiangmai, รู้ดี, มีประโยชน์, end]"
2,"[อายุ, 33, สูบบุหรี่, ดื่ม, เล่า, เครียด, นอน,..."
3,[end]
4,"[ตรวจ, หมอ, บอ, มะเร็ง, เม็ดเลือด, ชนิด, multi..."
...,...
149,[end]
150,"[พ่อ, ผม, อายุ, 70, อาการ, เสียงแหบ, เดือน, ตร..."
151,"[ผม, ทำ, แชร์, ตอน, ดี, มะ, end]"
152,[end]


In [35]:
fig = px.bar(word_count_table, x='count', y='word',text_auto=True,orientation='h')
fig.show()

In [51]:
from sentence_transformers import SentenceTransformer,util
import numpy as np
sentences = list(comment['comments'])
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embeddings = model.encode(sentences)
#Normalize the embeddings to unit length
Normalize_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
clustering_model = KMeans(n_clusters=4)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment)
print(len(cluster_assignment))

[3 1 3 3 2 2 2 2 2 0 3 3 3 2 2 3 3 2 2 3 0 3 1 2 0 2 2 3 3 3 2 0 2 3 3 1 2
 2 1 2 2 3 0 3 3 3 2 1 0 2 3 2 1 2 2 3 3 3 2 1 3 0 3 3 2 3 3 2 1 3 3 2 3 1
 3 1 0 3 0 3 2 2 3 3 3 0 3 2 0 3 3 2 1 0 3 2 0 2 0 3 1 1 2 3 2 2 3 3 3 3 2
 3 2 2 1 3 2 1 2 2 3 0 2 1 3 2 3 2 3 2 3 3 3 2 2 1 3 0 3 2 2 0 1 2 3 2 2 2
 3 2 3 0 3 0 3 3 1 3 1 3 0 3 0 0 3 3 3 2 2 1 2 3 2 3 3 0 1 2 2 2 3 0 0 0 2
 2 0 2 2 1 3 3 3 0 2 3 2 0 3 3 2 3 3 3 0 2 3 3 3 2 3 3 2 2 1 0 2 3 3 0 2 3
 1 3 0 3 0 3 1 3 3 0 3 1 3 2 2 2 2 3 2 0 1 3 3 3 0 1 2 2 2 0 3 2 3 3 3 2 3
 1 3 0 2 0 3 2 2 3 3 0 2 0 3 3 0 0 0 2 2 3 3 3 0 2 0 3 0 2 2 3 3 0]
292


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [55]:
rox= sentences
rox

['ขอแชร์เรื่องราวของแม่นะคะ (ตอนนี้กำลังทำการเข้าคีโม ครั้ง 6 จาก 12 ครั้ง ระยะเวลา 6 เดือน) #ไม่ได้ฉายแสง',
 'Mai Chiangmai ความรู้ดี มีประโยชน์',
 'ลูกพี่ลูกน้องเรา ตอนอายุ 28-29 ปีเป็นมะเร็งเต้านม หลังจากที่คลอดลูกสาวได้ 1-2 ปี มะเร็งเต้านมเป็นพันธุกรรมมาจากพี่สาวป้าเรา และพี่เราได้ตัดเต้านมทิ้ง... ดูเพิ่มเติม',
 'ตรวจพบว่าเป็นมะเร็งเต้านมระยะ 2 ตอนอายุ 28 ค่ะ ที่บ้านไม่มีใครเป็นเลย ไม่มีกรรมพันธุ์ใดๆ - พบก้อนครั้งแรกจากการคลำแล้วเจอก้อนแข็ง ไม่กลิ้ง ความรู้สึกเหมือนเป็นลูกแก้วหนึ่งลูก - ไปหาหมอ ทำอัลตราซาวน์บริเวณนม หลังจากนั้น หมอให้ทำแมมโมแกรม หลังจากแมมเสร็จ หมอสรุปว่ามีก้อนเนื้อ แต่ไม่รู้ว่าเนื้อร้ายหรือเนื้อดี ต้องเจาะชิ้นเนื้อไปตรวจ จึงพบว่าเป็นเนื้อร้าย ก็คือมะเร็งเต้านม - เราผ่าตัดแบบคว้านเอาเนื้อนมด้านในออกหมดเลย (เป็นนมซ้าย)- มีล่ามไปต่อมน้ำเหลืองด้วย ก็ต้องเอาต่อมน้ำเหลืองออกด้วย- หลังจากผ่าตัดเรียบร้อย รักษาแบบเต็มสตรีม ให้เคมีบำบัด ฉายรังสี และกินยา - เคมีบำบัด 16 ครั้ง แบบน้ำแดง 4 ครั้ง แบบน้ำขาว 12 ครั้ง - ฉายรังสี 25 ครั้ง - คุณหมอบอกให้กินยาต่ออีก 10 ปี เพราะเราเป็

In [46]:
clusternd_sentences= {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
  if cluster_id not in clusternd_sentences:
    clusternd_sentences[cluster_id] = []

  clusternd_sentences[cluster_id].append(sentences[sentence_id])

In [47]:
clusternd_sentences

{0: ['ขอแชร์เรื่องราวของแม่นะคะ (ตอนนี้กำลังทำการเข้าคีโม ครั้ง 6 จาก 12 ครั้ง ระยะเวลา 6 เดือน) #ไม่ได้ฉายแสง',
  'ลูกพี่ลูกน้องเรา ตอนอายุ 28-29 ปีเป็นมะเร็งเต้านม หลังจากที่คลอดลูกสาวได้ 1-2 ปี มะเร็งเต้านมเป็นพันธุกรรมมาจากพี่สาวป้าเรา และพี่เราได้ตัดเต้านมทิ้ง... ดูเพิ่มเติม',
  'ตรวจพบว่าเป็นมะเร็งเต้านมระยะ 2 ตอนอายุ 28 ค่ะ ที่บ้านไม่มีใครเป็นเลย ไม่มีกรรมพันธุ์ใดๆ - พบก้อนครั้งแรกจากการคลำแล้วเจอก้อนแข็ง ไม่กลิ้ง ความรู้สึกเหมือนเป็นลูกแก้วหนึ่งลูก - ไปหาหมอ ทำอัลตราซาวน์บริเวณนม หลังจากนั้น หมอให้ทำแมมโมแกรม หลังจากแมมเสร็จ หมอสรุปว่ามีก้อนเนื้อ แต่ไม่รู้ว่าเนื้อร้ายหรือเนื้อดี ต้องเจาะชิ้นเนื้อไปตรวจ จึงพบว่าเป็นเนื้อร้าย ก็คือมะเร็งเต้านม - เราผ่าตัดแบบคว้านเอาเนื้อนมด้านในออกหมดเลย (เป็นนมซ้าย)- มีล่ามไปต่อมน้ำเหลืองด้วย ก็ต้องเอาต่อมน้ำเหลืองออกด้วย- หลังจากผ่าตัดเรียบร้อย รักษาแบบเต็มสตรีม ให้เคมีบำบัด ฉายรังสี และกินยา - เคมีบำบัด 16 ครั้ง แบบน้ำแดง 4 ครั้ง แบบน้ำขาว 12 ครั้ง - ฉายรังสี 25 ครั้ง - คุณหมอบอกให้กินยาต่ออีก 10 ปี เพราะเราเป็นมะเร็งตอนอายุยังน้อย ',
  'ผลตรว